## Basic tests

In [5]:

from ruamel.yaml import StringIO
from six import StringIO

list1= ["Select asdasl"]
list2= "Select dasahdka"
list3= ["asdasda","asdasda"]


print(type(list1))
print(type(list2))
print(type(list3))

<class 'list'>
<class 'str'>
<class 'list'>


In [6]:
result = dc.query("SELECT count(column_name) as column_count FROM information_schema.columns WHERE table_name = 'test_table' and column_name != 'id'")[0][0]
print(f"--- {result}")

--- 0


In [7]:
result = dc.query("SELECT column_name as column_count FROM information_schema.columns WHERE table_name = 'test_table' and column_name != 'id'")
print(f">\n {result}")

list = tuple([column[0] for column in result])
print(list)

print(f"INSERT INTO table {list}")

>
 []
()
INSERT INTO table ()


In [8]:
return_value = [('name',), ('bezeichnung',)]
count = len(return_value)
print(count)

new_list = []
for i in return_value:
    new_list.append(i[0])

print(new_list)

new_list2 = [column[0] for column in return_value]
print(new_list2)

filtered = len(return_value)
print(f"({('%s,'*(filtered-1) + '%s')})")


2
['name', 'bezeichnung']
['name', 'bezeichnung']
(%s,%s)


In [9]:
values1 = None
values = ()

def ubs():
    if not values or len(values) < 1:
        return ("Please provide a 'value' when using this function")

ubs()

"Please provide a 'value' when using this function"

In [10]:
def generate_create_statement(table: str, values: tuple[tuple[str, str], ...]) -> str:
    columns = []
    for column_name, column_type in values:
        columns.append(f"{column_name} {column_type}")

    columns_sql = ",\n\t".join(columns)
    statement = f"CREATE TABLE {table} (\n\t{columns_sql}\n);"
    return statement


table = "ernst"
values = (
    ("id", "SERIAL PRIMARY KEY"),
    ("name", "varchar(255)"),
    ("type", "varchar(100)")
)

print(generate_create_statement(table,values))

CREATE TABLE ernst (
	id SERIAL PRIMARY KEY,
	name varchar(255),
	type varchar(100)
);


## Database tests
------------------------

In [1]:
import cloud_data.connectors.database_connector as dc

#### Query

In [2]:
statement = "SELECT * FROM test"
print(dc.query(statement))

[(4, 'Peter', 17), (6, 'Albert', 17), (7, 'Ivan', 22), (8, 'Ivan', 30), (9, 'Thomas', 34), (10, 'Stefan', 40), (11, 'Ivan', 22), (12, 'Ivan', 30), (13, 'Thomas', 34), (14, 'Stefan', 40), (15, 'Ivan', 23), (16, 'Ivan', 30), (17, 'Thomas', 34), (18, 'Stefan', 40), (19, 'Ivan', 30), (20, 'Thomas', 34), (21, 'Stefan', 40), (22, 'Ivan', 30), (23, 'Thomas', 34), (24, 'Stefan', 40), (25, 'Ivan', 30), (26, 'Thomas', 34), (27, 'Stefan', 40), (28, 'Ivan', 30), (29, 'Thomas', 34), (30, 'Stefan', 40), (31, 'Ivan', 30), (32, 'Thomas', 34), (33, 'Stefan', 40), (34, 'Ivan', 30), (35, 'Thomas', 34), (36, 'Stefan', 40), (37, 'Ivan', 30), (38, 'Thomas', 34), (39, 'Stefan', 40), (40, 'Ivan', 30), (41, 'Thomas', 34), (42, 'Stefan', 40), (43, 'asd', 12), (44, 'dfds', 345), (45, 'rzhtzbrr', 1231), (49, 'asd', 12), (50, 'dfds', 345), (51, 'rzhtzbrr', 1231), (52, 'asd', 12), (53, 'dfds', 345), (54, 'rzhtzbrr', 1231), (55, 'asd', 12), (56, 'dfds', 345), (57, 'rzhtzbrr', 1231), (58, 'asd', 12), (59, 'dfds', 345

#### Check Columns

In [3]:
tables = ["test","staging"]
dc.check_columns(tables[1])

[('col1',),
 ('col2',),
 ('col3',),
 ('col4',),
 ('col5',),
 ('col6',),
 ('col7',),
 ('col8',),
 ('col9',),
 ('processed',)]

#### _placeholder function

In [31]:
data1 = [('col1',), ('col2',), ('col3',), ('col4',), ('processed',)] # works db
data2 = [('name',), ('number',)] # works db
data3 = ["col001","col002","col003"] # works data
data4 = [('Ivan',30),('Thomas',34),('Stefan',40)]
data5 = [('Ivan',30)]

def _is_single_record(values):
    """
    Helper function that can be used to check if a list or tuple has multiple list/tuples inside.
    Will be used by the insert function to check if execute or executemany will be used.
    :param values: [tuple] / [list[tuple],[tuple]]
    :return: True or False
    """
    if not values:
        return True

    # Checks if values has a single element
    if isinstance(values, (list, tuple)):
        if len(values) == 0:
            return True

        first_element = values[0]

        if isinstance(first_element, (list, tuple)):
            return False
        else:
            return True

    return True

def _placeholder(value_list, input="db"):
    """
    Used as a helper function to get a nice string format of the provided list to be used in DB Statements.

    :param value_list: [('col1',),('col2')],["columnname1","columnname2"]
    :param input: db (default),data (for lists)
    :return: int 3, str (col1,col2), str (%s,%s)
    """
    is_single = _is_single_record(value_list)
    print(f"valuelist: {value_list}")
    print(f"valuelist[0]: {value_list[0]}")
    print(f"is_single: {is_single}")
    if input == "db":
        columns_tuple = tuple([column[0] for column in value_list])
    if input == "data":
        if is_single:
            columns_tuple = [str(value) for value in value_list]
            print(f"is_single"+ columns_tuple)
        else:
            columns_tuple = value_list[0]
            print("multiple items - " + columns_tuple[0])

    columns_count = len(columns_tuple)
    columns_string = "(" + ", ".join(columns_tuple) + ")"  # Values in the correct form for Statement (value1,value2)
    value_placeholder = f"({('%s,' * (columns_count - 1) + '%s')})"  # Creates (%s,%s,...) with the number of columns

    return columns_count, columns_string,value_placeholder, is_single

a,b,c,d = _placeholder(data5, input="data")

#a,b,c = dc._placeholder(data2, input="db")

print(a)
print(b)
print(c)

valuelist: [('Ivan', 30)]
valuelist[0]: ('Ivan', 30)
is_single: False
multiple items - Ivan


TypeError: sequence item 1: expected str instance, int found

In [28]:
data4 = [('Ivan',30),('Thomas',34),('Stefan',40)]
data5 = [('Ivan',30)]

print(data5[0])

('Ivan', 30)


#### Insert

##### .execute()

In [4]:
#Test mit .execute()

table = ["test","staging"]
values = [('Ivan',23), ("working","testscore",18,27,"Herbert","","","","",True),("faulty","testscore",17,25,"Herbert","","","",True)]
values_listoftuple = [("working","testscore3",18,27,"Herbert","","","","",True),("working","testscore2",18,27,"Herbert","","","","",True)]
# Table test hat die Columns "name", "number" ist alphabetisch
# Table staging id, col1-col15,processed


print(dc.insert(table=table[1],amount="single",values=values[1]))

INSERT INTO staging (col1, col2, col3, col4, col5, col6, col7, col8, col9, processed) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
INSERT 0 1


##### .executemany()

In [5]:
#Test mit .executemany()

table = "test"
# Table test hat die Columns "name", "number" ist alphabetisch
manyvalues = [('Ivan',30),('Thomas',34),('Stefan',40)]
print(dc.insert(table=table,amount="many",values=manyvalues))

INSERT INTO test (name, number) VALUES (%s,%s)
INSERT 0 1


#### COPY

In [3]:
filepath = "airflow_data/testdb.csv"
tablename = "test"
columns = ("name","number")
dc.copy_to_db(filepath,tablename,columns,format="CSV",header=True,delimiter=";")

File (airflow_data/testdb.csv) copied succesfully to table test.


In [2]:
columns,column_string,column_tuple, column_placeholder = dc._placeholder(dc.check_columns('staging_bbox'))
print(columns)
print(column_string)
print(column_tuple)
print(column_placeholder)

18
(training, epoch, current_time_start, current_time_end, mean_squared_error, mean_center_dist, mean_size_error, mean_overlap, combined_score, acc_all_conditions, train_loss, val_loss, train_acc, val_acc, current_lr, IMG_SIZE, BATCH_SIZE, processed)
('training', 'epoch', 'current_time_start', 'current_time_end', 'mean_squared_error', 'mean_center_dist', 'mean_size_error', 'mean_overlap', 'combined_score', 'acc_all_conditions', 'train_loss', 'val_loss', 'train_acc', 'val_acc', 'current_lr', 'IMG_SIZE', 'BATCH_SIZE', 'processed')
(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)


## Google Cloud Storage (Bucket)
--------------

### Initialize cloud_connector

In [1]:

from cloud_data.connectors import cloud_connector as cc

### Upload

In [4]:
# Full Filepath with filename
cc.upload_to_bucket("airflow_data/testdb.csv")

File airflow_data/testdb.csv uploaded as testdb.csv.


### Download

In [ ]:
# Filename in Bucket and Filelocation
cc.download_from_bucket("test25.txt","test25.txt")

## Airflow
---

In [1]:
import requests
from requests.auth import HTTPBasicAuth

username = "airflow"
password = "airflow"
airflow_endpoint = "http://localhost:8080/api/v1/"

#### Check Version

In [21]:
service = "version"
ENDPOINT_URL=airflow_endpoint + service
response = requests.get(ENDPOINT_URL)
print(response.json())

{'git_version': '.release:2.0.2+e494306fb01f3a026e7e2832ca94902e96b526fa', 'version': '2.0.2'}


#### List DAGs

In [5]:
service = "dags"
ENDPOINT_URL=airflow_endpoint + service

response = requests.get(ENDPOINT_URL,auth=HTTPBasicAuth(username, password))
if response.status_code == 200:
    print(response.json())
    print(response.json()['dags'][0]["dag_id"])
    dags_id = response.json()['dags'][0]["dag_id"]

else:
    print(response.status_code)
    print(response.text)

{'dags': [{'dag_id': 'Save_file_data_to_database', 'description': None, 'file_token': 'Ii9vcHQvYWlyZmxvdy9kYWdzL3BpcGVsaW5lX2RhdGFiYXNlLnB5Ig.2WY2_lM6bGCNVSBHY_HM8MQr27E', 'fileloc': '/opt/airflow/dags/pipeline_database.py', 'is_paused': True, 'is_subdag': False, 'owners': ['airflow'], 'root_dag_id': None, 'schedule_interval': {'__type': 'CronExpression', 'value': '*/5 * * * *'}, 'tags': []}], 'total_entries': 1}
Save_file_data_to_database


#### Information about DAG

In [6]:
service = f"dags/{dags_id}"
ENDPOINT_URL=airflow_endpoint + service

response = requests.get(ENDPOINT_URL,auth=HTTPBasicAuth(username, password))
if response.status_code == 200:
    print(response.json())
else:
    print(response.status_code)
    print(response.text)

{'dag_id': 'Save_file_data_to_database', 'description': None, 'file_token': 'Ii9vcHQvYWlyZmxvdy9kYWdzL3BpcGVsaW5lX2RhdGFiYXNlLnB5Ig.2WY2_lM6bGCNVSBHY_HM8MQr27E', 'fileloc': '/opt/airflow/dags/pipeline_database.py', 'is_paused': True, 'is_subdag': False, 'owners': ['airflow'], 'root_dag_id': None, 'schedule_interval': {'__type': 'CronExpression', 'value': '*/5 * * * *'}, 'tags': []}
